In [1]:
from config import *
args = get_args(args="--s_dims 64 --min_stddev 1e-5 --epochs 1 --load_name 20200430_001618 --load_epoch 1601 --device_ids None".split(), jupyter=True)
args.device_ids = []

In [2]:
ls models

20200430_001618/  20200501_163521/


In [3]:
args

Namespace(B=256, T=10, a_dim=4, comment=None, data_dir='~/tensorflow_datasets/', debug=False, device=0, device_ids=[], epochs=1, gamma=1e-05, h_dim=1024, load=True, load_dir='./models/20200430_001618', load_epoch=1601, load_name='20200430_001618', log_dir='./runs/20200502_155943', logfile='./logzero/20200502_155943.txt', loglevel=20, logzero_dir='./logzero/', min_stddev=1e-05, model='SSM', models_dir='./models/', name='20200502_155943', resnet=False, resume=False, runs_dir='./runs/', s_dims=[64], save_dir='./models/20200502_155943', seed=0, static_hierarchy=[])

In [37]:
from config import *
from model import *
from utils import *
from data_loader import PushDataLoader
from torch.utils.tensorboard import SummaryWriter
import torch
import logzero

logzero.loglevel(args.loglevel)

SEED = args.seed
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.autograd.set_detect_anomaly(True)

model = globals()[args.model](args)

train_loader = PushDataLoader(args, split="train", shuffle=True)
test_loader = PushDataLoader(args, split="test", shuffle=False)

if args.load or args.resume:
    load_model(model, args.load_dir, args.load_epoch, load_optim=args.resume)

for i, batch in enumerate(test_loader):
    x, a, itr = batch
    _B = x.size(0)
    x = x.transpose(0, 1).to(args.device)  # T,B,3,28,28
    a = a.transpose(0, 1).to(args.device)  # T,B,1
    x_0 = x[0].clone()
    x_ = model.sample_x(x_0, x, a)

In [34]:
import matplotlib.pyplot as plt
import numpy as np
import moviepy.editor as mpy


def npy_to_gif(npy, filename):
    clip = mpy.ImageSequenceClip(list(npy), fps=10)
    clip.write_gif(filename)

x_true = []
for i in range(8):
    xx = x_[0+i][:].detach().cpu().numpy().transpose([0,2,3,1])
    xx = (xx * 255).astype(np.uint8)
    x_true.append(xx)

x_true = np.concatenate(x_true, 2)

x_pred = []
for i in range(8):
    xx = x_[8+i][:].detach().cpu().numpy().transpose([0,2,3,1])
    xx = (xx * 255).astype(np.uint8)
    x_pred.append(xx)

x_pred = np.concatenate(x_pred, 2)

npy = np.concatenate([x_true, x_pred], 1)

npy_to_gif(npy, "sample.gif")

t:   0%|          | 0/10 [00:00<?, ?it/s, now=None]

MoviePy - Building file sample.gif with imageio.
